In [59]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [60]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [61]:
tf.keras.backend.clear_session()

In [62]:
subject = 'Minas Gerais - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [63]:
data = pd.read_csv('2003_mo_model_input_MG.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
# data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Minas Gerais - IDH,Minas Gerais - IDH Educacao,Minas Gerais - value,Minas Gerais - IDH Renda,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),Direcionamento Construção (R$),Direcionamento Aplicação (R$),Ativos Problemáticos - Home equity (R$),Ativos Problemáticos - SFH (R$),Ativos Problemáticos - FGTS (R$),Carteira de Crédito – Home equity (R$),Carteira de Crédito – Livre (R$),Carteira de Crédito - FGTS (R$),Carteira de Crédito - SFH (R$),Carteira de Crédito P.Jurídica - Comercial (R$),Carteira de Crédito P.Jurídica - Livre (R$),Carteira de Crédito P.Jurídica - FGTS (R$),Inadimplência - SFH (%),Inadimplência - FGTS (%),Inadimplência - Livre (%),Inadimplência - Comercial (%),Inadimplência - Home equity (%),Poupança e Empréstimo - SBPE (R$),Títulos - LH (R$),Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - IDH Longevidade,Minas Gerais - Consumo de Cimento (t)
0,2003-1,0.756537,0.698929,0.334536,0.759345,648.891160,8.242462,2.937752e+08,1.725077e+07,14.096054,2.759117e+08,2.014625e+08,1.542506e+09,9.248956e+08,3.117876e+09,4.235269e+08,5.871672e+08,1.300935e+11,1.039573e+10,1.145754e+11,4.355660e+08,3.647348e+09,4.601248e+09,9.143676e+09,7.702509e+10,6.664510e+10,7.035792e+10,4.872928e+08,1.970044e+10,2.021690e+10,1.557727,2.468118,1.603214,0.300795,3.988206,1.540380e+11,1.211245e+09,8.107262e+09,1.921789e+10,8.202881e+10,0.724032,-5331.049150,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.826175,247.134
1,2003-2,0.756708,0.699250,0.335670,0.759119,652.396314,8.236470,2.941480e+08,1.726959e+07,14.102603,2.760648e+08,2.015196e+08,1.542506e+09,9.267621e+08,3.118866e+09,4.235269e+08,5.871672e+08,1.300935e+11,1.039573e+10,1.145754e+11,4.373259e+08,3.662680e+09,4.601248e+09,9.149335e+09,7.702509e+10,6.664510e+10,7.035792e+10,4.872928e+08,1.970044e+10,2.021690e+10,1.557727,2.467045,1.603118,0.300795,3.991229,1.540380e+11,1.211245e+09,8.107262e+09,1.921789e+10,8.222716e+10,0.690297,-5318.079644,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.826617,305.708
2,2003-3,0.756879,0.699571,0.336553,0.758894,655.911573,8.230479,2.945208e+08,1.728841e+07,14.109152,2.762178e+08,2.015529e+08,1.542506e+09,9.290952e+08,3.119443e+09,4.235269e+08,5.871672e+08,1.300935e+11,1.039573e+10,1.145754e+11,4.383525e+08,3.671623e+09,4.601248e+09,9.152636e+09,7.702509e+10,6.664510e+10,7.035792e+10,4.872928e+08,1.970044e+10,2.021690e+10,1.557727,2.466420,1.603063,0.300795,3.992993,1.540380e+11,1.211245e+09,8.107262e+09,1.921789e+10,8.234287e+10,0.669681,-5436.417870,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.827060,284.168
3,2003-4,0.757050,0.699892,0.336531,0.758669,656.746430,8.224488,2.948935e+08,1.730723e+07,14.115701,2.763709e+08,2.015565e+08,1.542506e+09,9.304019e+08,3.119505e+09,4.235269e+08,5.871672e+08,1.300935e+11,1.039573e+10,1.145754e+11,4.384625e+08,3.672581e+09,4.601248e+09,9.152990e+09,7.702509e+10,6.664510e+10,7.035792e+10,4.872928e+08,1.970044e+10,2.021690e+10,1.557727,2.466353,1.603057,0.300795,3.993182,1.540380e+11,1.211245e+09,8.107262e+09,1.921789e+10,8.235526e+10,0.660494,-5707.015274,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.827502,286.594
4,2003-5,0.757222,0.700213,0.336406,0.758443,660.114672,8.218497,2.952663e+08,1.732605e+07,14.122250,2.765239e+08,2.017543e+08,1.544532e+09,9.329889e+08,3.125213e+09,4.241975e+08,5.877836e+08,1.301077e+11,1.039463e+10,1.147057e+11,4.391591e+08,3.

In [64]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Minas Gerais - IDH,Minas Gerais - IDH Educacao,Minas Gerais - value,Minas Gerais - IDH Renda,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),Direcionamento Construção (R$),Direcionamento Aplicação (R$),Ativos Problemáticos - Home equity (R$),Ativos Problemáticos - SFH (R$),Ativos Problemáticos - FGTS (R$),Carteira de Crédito – Home equity (R$),Carteira de Crédito – Livre (R$),Carteira de Crédito - FGTS (R$),Carteira de Crédito - SFH (R$),Carteira de Crédito P.Jurídica - Comercial (R$),Carteira de Crédito P.Jurídica - Livre (R$),Carteira de Crédito P.Jurídica - FGTS (R$),Inadimplência - SFH (%),Inadimplência - FGTS (%),Inadimplência - Livre (%),Inadimplência - Comercial (%),Inadimplência - Home equity (%),Poupança e Empréstimo - SBPE (R$),Títulos - LH (R$),Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - IDH Longevidade
0,-1.917914,-1.774202,-1.524203,1.929432,-2.146709,-0.699400,-1.745399,-1.973963,-3.018821,-2.686146,-1.329044,-0.409905,-1.240817,-1.095326,-0.230491,-0.194106,-1.022744,-0.900964,-1.382302,-0.536376,-1.414047,-1.088562,-1.252067,1.768350,-0.858773,-1.173480,-1.737283,-0.545087,-0.839415,1.416258,0.679015,1.062105,-1.190708,-0.026995,-1.615339,-0.300935,1.922051,-1.023713,-0.767541,2.723741,0.364808,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.448814
1,-1.892160,-1.754356,-1.508666,1.904269,-2.125900,-0.706157,-1.725016,-1.923436,-2.934320,-2.623049,-1.327215,-0.409905,-1.239136,-1.094441,-0.230491,-0.194106,-1.022744,-0.900964,-1.382302,-0.532013,-1.407886,-1.088562,-1.246382,1.768350,-0.858773,-1.173480,-1.737283,-0.545087,-0.839415,1.416258,0.674479,1.061740,-1.190708,-0.023784,-1.615339,-0.300935,1.922051,-1.023713,-0.762847,2.350880,0.368956,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.401003
2,-1.866406,-1.734511,-1.496577,1.879107,-2.105031,-0.712914,-1.704633,-1.872910,-2.849818,-2.559951,-1.326148,-0.409905,-1.237035,-1.093925,-0.230491,-0.194106,-1.022744,-0.900964,-1.382302,-0.529468,-1.404292,-1.088562,-1.243066,1.768350,-0.858773,-1.173480,-1.737283,-0.545087,-0.839415,1.416258,0.671833,1.061528,-1.190708,-0.021910,-1.615339,-0.300935,1.922051,-1.023713,-0.760108,2.123016,0.331110,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.353192
3,-1.840653,-1.714666,-1.496879,1.853945,-2.100075,-0.719672,-1.684250,-1.822384,-2.765317,-2.496853,-1.326033,-0.409905,-1.235858,-1.093869,-0.230491,-0.194106,-1.022744,-0.900964,-1.382302,-0.529195,-1.403907,-1.088562,-1.242711,1.768350,-0.858773,-1.173480,-1.737283,-0.545087,-0.839415,1.416258,0.671549,1.061505,-1.190708,-0.021709,-1.615339,-0.300935,1.922051,-1.023713,-0.759815,2.021477,0.244570,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.305382
4,-1.814899,-1.694821,-1.498597,1.828782,-2.080078,-0.726429,-1.663867,-1.771858,-2.680816,-2.433756,-1.319698,-0.406927,-1.233528,-1.088767,-0.227470,-0.190987,-1.022627,-0.901084,-1.381163,-0.527468,-1.398168,-1.088024,-1.238181,1.764433,-0.858957,-1.173107,-1.733705,-0.544159,-0.837772,1.411669,0.669072,1.056392,-1.190038,-0.022002,-1.613410,-0.298382,1.916280,-1.023685,-0.757341,1.887113,0.279013,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.257571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.315321,1.407871,0.874645,-1.328539,

In [65]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      249.680
1      219.561
2      304.641
3      262.564
4      299.209
        ...   
236        NaN
237        NaN
238        NaN
239        NaN
240        NaN
Name: Minas Gerais - Consumo de Cimento (t), Length: 241, dtype: float64

In [66]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Minas Gerais - IDH,Minas Gerais - IDH Educacao,Minas Gerais - value,Minas Gerais - IDH Renda,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),Direcionamento Construção (R$),Direcionamento Aplicação (R$),Ativos Problemáticos - Home equity (R$),Ativos Problemáticos - SFH (R$),Ativos Problemáticos - FGTS (R$),Carteira de Crédito – Home equity (R$),Carteira de Crédito – Livre (R$),Carteira de Crédito - FGTS (R$),Carteira de Crédito - SFH (R$),Carteira de Crédito P.Jurídica - Comercial (R$),Carteira de Crédito P.Jurídica - Livre (R$),Carteira de Crédito P.Jurídica - FGTS (R$),Inadimplência - SFH (%),Inadimplência - FGTS (%),Inadimplência - Livre (%),Inadimplência - Comercial (%),Inadimplência - Home equity (%),Poupança e Empréstimo - SBPE (R$),Títulos - LH (R$),Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - IDH Longevidade
0,-1.917914,-1.774202,-1.524203,1.929432,-2.146709,-0.699400,-1.745399,-1.973963,-3.018821,-2.686146,-1.329044,-0.409905,-1.240817,-1.095326,-0.230491,-0.194106,-1.022744,-0.900964,-1.382302,-0.536376,-1.414047,-1.088562,-1.252067,1.768350,-0.858773,-1.173480,-1.737283,-0.545087,-0.839415,1.416258,0.679015,1.062105,-1.190708,-0.026995,-1.615339,-0.300935,1.922051,-1.023713,-0.767541,2.723741,0.364808,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.448814
1,-1.892160,-1.754356,-1.508666,1.904269,-2.125900,-0.706157,-1.725016,-1.923436,-2.934320,-2.623049,-1.327215,-0.409905,-1.239136,-1.094441,-0.230491,-0.194106,-1.022744,-0.900964,-1.382302,-0.532013,-1.407886,-1.088562,-1.246382,1.768350,-0.858773,-1.173480,-1.737283,-0.545087,-0.839415,1.416258,0.674479,1.061740,-1.190708,-0.023784,-1.615339,-0.300935,1.922051,-1.023713,-0.762847,2.350880,0.368956,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.401003
2,-1.866406,-1.734511,-1.496577,1.879107,-2.105031,-0.712914,-1.704633,-1.872910,-2.849818,-2.559951,-1.326148,-0.409905,-1.237035,-1.093925,-0.230491,-0.194106,-1.022744,-0.900964,-1.382302,-0.529468,-1.404292,-1.088562,-1.243066,1.768350,-0.858773,-1.173480,-1.737283,-0.545087,-0.839415,1.416258,0.671833,1.061528,-1.190708,-0.021910,-1.615339,-0.300935,1.922051,-1.023713,-0.760108,2.123016,0.331110,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.353192
3,-1.840653,-1.714666,-1.496879,1.853945,-2.100075,-0.719672,-1.684250,-1.822384,-2.765317,-2.496853,-1.326033,-0.409905,-1.235858,-1.093869,-0.230491,-0.194106,-1.022744,-0.900964,-1.382302,-0.529195,-1.403907,-1.088562,-1.242711,1.768350,-0.858773,-1.173480,-1.737283,-0.545087,-0.839415,1.416258,0.671549,1.061505,-1.190708,-0.021709,-1.615339,-0.300935,1.922051,-1.023713,-0.759815,2.021477,0.244570,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.305382
4,-1.814899,-1.694821,-1.498597,1.828782,-2.080078,-0.726429,-1.663867,-1.771858,-2.680816,-2.433756,-1.319698,-0.406927,-1.233528,-1.088767,-0.227470,-0.190987,-1.022627,-0.901084,-1.381163,-0.527468,-1.398168,-1.088024,-1.238181,1.764433,-0.858957,-1.173107,-1.733705,-0.544159,-0.837772,1.411669,0.669072,1.056392,-1.190038,-0.022002,-1.613410,-0.298382,1.916280,-1.023685,-0.757341,1.887113,0.279013,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.257571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.399834,1.355364,0.832198,-1.319310,

In [67]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      249.680
1      219.561
2      304.641
3      262.564
4      299.209
        ...   
157    338.767
158    419.962
159    387.800
160    391.122
161    409.739
Name: Minas Gerais - Consumo de Cimento (t), Length: 162, dtype: float64

In [68]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [69]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 48)

In [70]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Minas Gerais - IDH,Minas Gerais - IDH Educacao,Minas Gerais - value,Minas Gerais - IDH Renda,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),Direcionamento Construção (R$),Direcionamento Aplicação (R$),Ativos Problemáticos - Home equity (R$),Ativos Problemáticos - SFH (R$),Ativos Problemáticos - FGTS (R$),Carteira de Crédito – Home equity (R$),Carteira de Crédito – Livre (R$),Carteira de Crédito - FGTS (R$),Carteira de Crédito - SFH (R$),Carteira de Crédito P.Jurídica - Comercial (R$),Carteira de Crédito P.Jurídica - Livre (R$),Carteira de Crédito P.Jurídica - FGTS (R$),Inadimplência - SFH (%),Inadimplência - FGTS (%),Inadimplência - Livre (%),Inadimplência - Comercial (%),Inadimplência - Home equity (%),Poupança e Empréstimo - SBPE (R$),Títulos - LH (R$),Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - IDH Longevidade
126,0.544198,0.555372,1.135116,-0.656829,0.964210,1.008346,0.811298,0.287865,0.502913,0.863809,0.561200,0.469010,0.654287,0.514061,0.169137,-0.058348,0.381693,0.263992,0.600448,0.384917,0.576969,0.265242,0.888320,-0.202741,0.191660,0.602105,0.155824,1.350509,1.172681,-0.264671,-0.194824,-0.162207,0.453853,0.188388,0.773189,0.592023,-0.689873,0.403484,0.557912,0.888984,-0.230612,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.344129
127,0.559144,0.573759,1.150078,-0.649246,0.924638,1.051039,0.826912,0.237130,0.464014,0.848131,0.487970,0.414665,0.688711,0.442312,0.103544,-0.133421,0.412779,0.295413,0.619650,0.447173,0.598465,0.285110,0.933789,-0.155709,0.221973,0.636016,0.111878,1.373026,1.179922,-0.189339,-0.240395,-0.071411,0.479039,0.262788,0.782160,0.550513,-0.689873,0.436846,0.616673,0.954254,-0.156322,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.340546
128,0.574090,0.592145,1.162632,-0.641662,0.882104,1.093731,0.842525,0.186395,0.425115,0.832454,0.411684,0.358242,0.723533,0.367653,0.035698,-0.210949,0.444315,0.327387,0.638700,0.511105,0.619935,0.305090,0.979938,-0.106412,0.252850,0.670325,0.065792,1.395529,1.186617,-0.110946,-0.287128,0.022759,0.504395,0.339514,0.790494,0.507329,-0.689873,0.470723,0.676845,1.045217,-0.120008,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.336963
129,0.589036,0.610532,1.166985,-0.634079,0.846537,1.136424,0.858138,0.135660,0.386216,0.816777,0.332303,0.299713,0.758754,0.290047,-0.034429,-0.290961,0.476306,0.359921,0.657591,0.576734,0.641378,0.325180,1.026772,-0.054817,0.284295,0.705035,0.017540,1.418016,1.192754,-0.029453,-0.335037,0.120347,0.529920,0.418596,0.798179,0.462450,-0.689873,0.505118,0.738444,1.176395,-0.143194,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.333380
130,0.603982,0.628918,1.145581,-0.626496,0.796933,1.179116,0.873751,0.084924,0.347317,0.801100,0.249788,0.239053,0.794376,0.209457,-0.106863,-0.373485,0.508754,0.393019,0.676320,0.644076,0.662791,0.345382,1.074297,-0.000896,0.316313,0.740146,-0.032908,1.440483,1.198325,0.055179,-0.384135,0.221394,0.555614,0.500061,0.805204,0.415855,-0.689873,0.540037,0.801485,1.303259,-0.028712,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.329797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.315321,1.407871,0.874645,-1.328539,0.774558,0.789659,1.149269,-0.772657,0.183801,0.694771,-0.124730,-1.726985,1.267258,-0.274933,-1.288888,-0.794961,2.

In [71]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 48)

In [72]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [73]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(180, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(108, return_sequences=True, activation='tanh'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(36, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [74]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [75]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[212943067, 3907499616, 880225805, 143189312, 3188706932, 3403511122, 3482804653, 2082278014, 2635488858, 2454438727, 2730257132, 1903995920, 3592962565, 4182181004, 1257287439, 3448286349, 3473925502, 2575601461, 670081447, 2525822336, 3425283816, 2845809902, 3155993933, 881383572, 263805305, 185656332, 2610041437, 2229644746, 1851339637, 684751160, 1274135579, 1871571705, 512412937, 2730851826, 3347998766, 2363258524, 1541591207, 1863945896, 2979697306, 3588294576, 576702493, 3817829082, 1674650882, 3012172691, 1423667069, 312776175, 2014097835, 1808940156, 651863562, 3280081937]


Step: 0 ___________________________________________
val_loss: 1265.0750732421875
winner_seed: 212943067


Step: 1 ___________________________________________
val_loss: 2445.46484375


Step: 2 ___________________________________________
val_loss: 2593.087646484375


Step: 3 ___________________________________________
val_loss: 7576.6796875


Step: 4 ___________________________________________
val_loss: 2449

In [76]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 185ms/step - loss: 255449.1406 - val_loss: 73813.0391
Epoch 2/10000
4/4 [==============================] - 0s 15ms/step - loss: 43138.1680 - val_loss: 11693.3633
Epoch 3/10000
4/4 [==============================] - 0s 16ms/step - loss: 9741.9766 - val_loss: 7675.2378
Epoch 4/10000
4/4 [==============================] - 0s 16ms/step - loss: 7709.1553 - val_loss: 7821.9946
Epoch 5/10000
4/4 [==============================] - 0s 15ms/step - loss: 7644.9644 - val_loss: 47912.8125
Epoch 6/10000
4/4 [==============================] - 0s 15ms/step - loss: 16265.2051 - val_loss: 12468.6709
Epoch 7/10000
4/4 [==============================] - 0s 14ms/step - loss: 9906.6123 - val_loss: 7873.5356
Epoch 8/10000
4/4 [==============================] - 0s 16ms/step - loss: 7581.7681 - val_loss: 7275.3770
Epoch 9/10000
4/4 [==============================] - 0s 15ms/step - loss: 6738.4590 - val_loss: 7058.3232
Epoch 10/10000
4/4 [=================

4/4 [==============================] - 0s 14ms/step - loss: 5962.3237 - val_loss: 5791.4165
Epoch 78/10000
4/4 [==============================] - 0s 16ms/step - loss: 5968.0498 - val_loss: 5569.0537
Epoch 79/10000
4/4 [==============================] - 0s 16ms/step - loss: 5861.6396 - val_loss: 5574.0879
Epoch 80/10000
4/4 [==============================] - 0s 15ms/step - loss: 5916.9644 - val_loss: 5582.4199
Epoch 81/10000
4/4 [==============================] - 0s 16ms/step - loss: 5930.4224 - val_loss: 5520.4824
Epoch 82/10000
4/4 [==============================] - 0s 16ms/step - loss: 6049.7651 - val_loss: 5545.6226
Epoch 83/10000
4/4 [==============================] - 0s 15ms/step - loss: 5926.1855 - val_loss: 5520.6255
Epoch 84/10000
4/4 [==============================] - 0s 14ms/step - loss: 5801.1475 - val_loss: 5486.7544
Epoch 85/10000
4/4 [==============================] - 0s 16ms/step - loss: 5863.6929 - val_loss: 5490.9907
Epoch 86/10000
4/4 [==============================] 

4/4 [==============================] - 0s 14ms/step - loss: 3293.4263 - val_loss: 3433.4783
Epoch 154/10000
4/4 [==============================] - 0s 16ms/step - loss: 3338.8403 - val_loss: 3849.0881
Epoch 155/10000
4/4 [==============================] - 0s 16ms/step - loss: 3719.0645 - val_loss: 4100.4614
Epoch 156/10000
4/4 [==============================] - 0s 17ms/step - loss: 3679.5574 - val_loss: 4175.4395
Epoch 157/10000
4/4 [==============================] - 0s 17ms/step - loss: 3807.2969 - val_loss: 3497.7341
Epoch 158/10000
4/4 [==============================] - 0s 17ms/step - loss: 4180.8335 - val_loss: 3772.9463
Epoch 159/10000
4/4 [==============================] - 0s 16ms/step - loss: 3453.3406 - val_loss: 3688.7136
Epoch 160/10000
4/4 [==============================] - 0s 16ms/step - loss: 3238.2051 - val_loss: 4212.8682
Epoch 161/10000
4/4 [==============================] - 0s 15ms/step - loss: 2775.5203 - val_loss: 3556.3218
Epoch 162/10000
4/4 [=======================

4/4 [==============================] - 0s 12ms/step - loss: 5751.7744 - val_loss: 5484.8730
Epoch 230/10000
4/4 [==============================] - 0s 12ms/step - loss: 4281.9834 - val_loss: 5588.0898
Epoch 231/10000
4/4 [==============================] - 0s 11ms/step - loss: 4176.4307 - val_loss: 6119.8599
Epoch 232/10000
4/4 [==============================] - 0s 11ms/step - loss: 5056.2832 - val_loss: 3920.7539
Epoch 233/10000
4/4 [==============================] - 0s 12ms/step - loss: 4017.6130 - val_loss: 3433.9453
Epoch 234/10000
4/4 [==============================] - 0s 12ms/step - loss: 3952.9041 - val_loss: 4655.1880
Epoch 235/10000
4/4 [==============================] - 0s 13ms/step - loss: 3799.8682 - val_loss: 3604.9463
Epoch 236/10000
4/4 [==============================] - 0s 13ms/step - loss: 3254.8843 - val_loss: 4599.0156
Epoch 237/10000
4/4 [==============================] - 0s 13ms/step - loss: 2971.2002 - val_loss: 4392.1001
Epoch 238/10000
4/4 [=======================

4/4 [==============================] - 0s 11ms/step - loss: 3540.6702 - val_loss: 3719.6436
Epoch 306/10000
4/4 [==============================] - 0s 12ms/step - loss: 2784.2593 - val_loss: 3476.5745
Epoch 307/10000
4/4 [==============================] - 0s 11ms/step - loss: 2522.2869 - val_loss: 3492.1338
Epoch 308/10000
4/4 [==============================] - 0s 12ms/step - loss: 2758.6833 - val_loss: 3128.2756
Epoch 309/10000
4/4 [==============================] - 0s 12ms/step - loss: 3007.5322 - val_loss: 3903.8142
Epoch 310/10000
4/4 [==============================] - 0s 12ms/step - loss: 3300.5830 - val_loss: 3658.8096
Epoch 311/10000
4/4 [==============================] - 0s 12ms/step - loss: 2801.6802 - val_loss: 3616.1128
Epoch 312/10000
4/4 [==============================] - 0s 12ms/step - loss: 2470.0542 - val_loss: 5477.8115
Epoch 313/10000
4/4 [==============================] - 0s 12ms/step - loss: 3594.3850 - val_loss: 4680.5913
Epoch 314/10000
4/4 [=======================

...

4/4 [==============================] - 0s 16ms/step - loss: 1968.4884 - val_loss: 3442.0486
Epoch 990/10000
4/4 [==============================] - 0s 16ms/step - loss: 1860.3488 - val_loss: 3353.7307
Epoch 991/10000
4/4 [==============================] - 0s 15ms/step - loss: 2115.5581 - val_loss: 3519.8975
Epoch 992/10000
4/4 [==============================] - 0s 17ms/step - loss: 1945.1464 - val_loss: 3175.2505
Epoch 993/10000
4/4 [==============================] - 0s 17ms/step - loss: 1869.0851 - val_loss: 3180.6121
Epoch 994/10000
4/4 [==============================] - 0s 17ms/step - loss: 1921.5443 - val_loss: 3192.3486
Epoch 995/10000
4/4 [==============================] - 0s 17ms/step - loss: 1927.9781 - val_loss: 3135.0408
Epoch 996/10000
4/4 [==============================] - 0s 17ms/step - loss: 1939.3566 - val_loss: 3310.7512
Epoch 997/10000
4/4 [==============================] - 0s 16ms/step - loss: 2001.5240 - val_loss: 2932.1299
Epoch 998/10000
4/4 [=======================

4/4 [==============================] - 0s 16ms/step - loss: 1978.9236 - val_loss: 3372.9482
Epoch 1065/10000
4/4 [==============================] - 0s 15ms/step - loss: 1792.6501 - val_loss: 3382.5735
Epoch 1066/10000
4/4 [==============================] - 0s 17ms/step - loss: 2096.8076 - val_loss: 3391.9653
Epoch 1067/10000
4/4 [==============================] - 0s 16ms/step - loss: 2210.2651 - val_loss: 3483.3306
Epoch 1068/10000
4/4 [==============================] - 0s 15ms/step - loss: 2160.8831 - val_loss: 3509.8154
Epoch 1069/10000
4/4 [==============================] - 0s 15ms/step - loss: 2060.8208 - val_loss: 3564.1748
Epoch 1070/10000
4/4 [==============================] - 0s 17ms/step - loss: 2081.7898 - val_loss: 3479.0828
Epoch 1071/10000
4/4 [==============================] - 0s 17ms/step - loss: 2193.6025 - val_loss: 3447.0161
Epoch 1072/10000
4/4 [==============================] - 0s 17ms/step - loss: 1996.5203 - val_loss: 3168.1970
Epoch 1073/10000
4/4 [==============

4/4 [==============================] - 0s 14ms/step - loss: 2304.9363 - val_loss: 3302.4355
Epoch 1140/10000
4/4 [==============================] - 0s 16ms/step - loss: 2112.7417 - val_loss: 3144.4199
Epoch 1141/10000
4/4 [==============================] - 0s 18ms/step - loss: 2348.6472 - val_loss: 3016.0339
Epoch 1142/10000
4/4 [==============================] - 0s 16ms/step - loss: 2253.9019 - val_loss: 3265.3608
Epoch 1143/10000
4/4 [==============================] - 0s 15ms/step - loss: 2247.7336 - val_loss: 3427.8865
Epoch 1144/10000
4/4 [==============================] - 0s 16ms/step - loss: 1952.2526 - val_loss: 3367.4670
Epoch 1145/10000
4/4 [==============================] - 0s 15ms/step - loss: 2202.7847 - val_loss: 3530.9036
Epoch 1146/10000
4/4 [==============================] - 0s 15ms/step - loss: 2032.7164 - val_loss: 3401.9670
Epoch 1147/10000
4/4 [==============================] - 0s 16ms/step - loss: 2164.9121 - val_loss: 2996.7300
Epoch 1148/10000
4/4 [==============

4/4 [==============================] - 0s 16ms/step - loss: 2055.1887 - val_loss: 4174.4302
Epoch 1215/10000
4/4 [==============================] - 0s 16ms/step - loss: 2186.7754 - val_loss: 4098.9995
Epoch 1216/10000
4/4 [==============================] - 0s 17ms/step - loss: 2203.1182 - val_loss: 4073.1016
Epoch 1217/10000
4/4 [==============================] - 0s 17ms/step - loss: 2106.4341 - val_loss: 4114.2725
Epoch 1218/10000
4/4 [==============================] - 0s 16ms/step - loss: 2141.8533 - val_loss: 3860.2109
Epoch 1219/10000
4/4 [==============================] - 0s 16ms/step - loss: 1926.1671 - val_loss: 3739.0715
Epoch 1220/10000
4/4 [==============================] - 0s 17ms/step - loss: 2179.6824 - val_loss: 3468.8057
Epoch 1221/10000
4/4 [==============================] - 0s 17ms/step - loss: 2257.5793 - val_loss: 3682.3684
Epoch 1222/10000
4/4 [==============================] - 0s 15ms/step - loss: 2113.1748 - val_loss: 3635.9136
Epoch 1223/10000
4/4 [==============

4/4 [==============================] - 0s 15ms/step - loss: 1831.3038 - val_loss: 3347.7527
Epoch 1290/10000
4/4 [==============================] - 0s 14ms/step - loss: 2172.6958 - val_loss: 3526.1592
Epoch 1291/10000
4/4 [==============================] - 0s 16ms/step - loss: 1840.9185 - val_loss: 3299.9773
Epoch 1292/10000
4/4 [==============================] - 0s 16ms/step - loss: 1887.6541 - val_loss: 4220.8438
Epoch 1293/10000
4/4 [==============================] - 0s 16ms/step - loss: 1958.3136 - val_loss: 4590.3789
Epoch 1294/10000
4/4 [==============================] - 0s 17ms/step - loss: 2251.9370 - val_loss: 3675.0012
Epoch 1295/10000
4/4 [==============================] - 0s 17ms/step - loss: 1883.1779 - val_loss: 3117.8672
Epoch 1296/10000
4/4 [==============================] - 0s 17ms/step - loss: 2114.4065 - val_loss: 3525.6938
Epoch 1297/10000
4/4 [==============================] - 0s 15ms/step - loss: 1892.1461 - val_loss: 3512.3533
Epoch 1298/10000
4/4 [==============

In [77]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [78]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 24ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,414.416809,414.3815,416.613068,414.806793,419.385162,421.038208,417.010681,414.608154,414.785126,414.464264,418.551483,415.768097,416.315796,425.792358,410.967194,445.954468,449.973511,445.117889,469.524506,415.233887,413.24176,410.290619,406.620392,403.396606,407.0047,400.746277,409.48465,402.694611,408.850952,413.129608
Target,413.46,497.019,476.478,421.395,413.641,338.033,431.028,305.105,402.406,419.419,336.054,568.395,541.488,537.851,526.265,559.293,466.646,416.049,501.489,460.699,445.583,509.271,532.602,500.905,588.495,589.349,543.982,614.074,557.419,412.582
Error,0.956818,82.637512,59.864929,6.588196,5.744171,83.005219,14.017334,109.503143,12.37912,4.954742,82.497498,152.626923,125.17218,112.058655,115.297821,113.338562,16.672485,29.068878,31.964508,45.465118,32.341248,98.980377,125.981598,97.508392,181.490295,188.602722,134.497345,211.379364,148.568054,0.547607


In [79]:
display(mae)
display(mape)

80.79037

0.16108193

In [80]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [81]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined